In [1]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR, CONFIG_DIR
from omegaconf import OmegaConf
from src.external.hptr.src.data_modules.agent_centric import AgentCentricPreProcessing
from src.external.hptr.src.data_modules.ac_global import AgentCentricGlobal
from src.mimolm import InputProjections, EarlyFusionEncoder, MotionDecoder
import torch

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

2025-01-25 18:43:41.026 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: G:\Work\DS\mimolm


g:\Work\DS\mimolm\notebooks


In [2]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2

data_module = DataH5av2(DATA_DIR / "interim" / "train" )
data_module.setup(stage="fit")
dl = data_module.train_dataloader()

c:\Users\Harshavardhan Patil\.virtualenvs\mimolm-kv68Yv56\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#dic = OmegaConf.load(Path(CONFIG_DIR / "model/ac_wayformer.yaml"))
episode = None
fmask = None
prepreprocess = AgentCentricPreProcessing(time_step_current=49, 
                                          data_size=data_module.tensor_size_train, 
                                          n_target=8,
                                          n_other=48,
                                          n_map=512,
                                          mask_invalid=False)

preprocess = AgentCentricGlobal(time_step_current=49,
                                data_size=data_module.tensor_size_train,
                                dropout_p_history=0.15, 
                                add_ohe=True,
                                pl_aggr=False,
                                pose_pe= {"agent": "xy_dir",
                                        "map": "mpa_pl"})

inputproj = InputProjections(hidden_dim=256,
                             agent_attr_dim=preprocess.model_kwargs["agent_attr_dim"],
                             map_attr_dim=preprocess.model_kwargs["map_attr_dim"],
                             n_step_hist=preprocess.model_kwargs["n_step_hist"],
                             n_pl_node=preprocess.model_kwargs["n_pl_node"],
                             add_learned_pe=True,
                             n_layer_mlp=3,
                             mlp_cfg={"end_layer_activation" : True,
                                        "use_layernorm" :  False,
                                        "use_batchnorm" : False,
                                        "dropout_p" : None,})

encoder = EarlyFusionEncoder(hidden_dim=256,
                             tf_cfg={"n_head": 4,
                                     "dropout_p": 0.1,
                                     "norm_first": True,
                                     "bias": True},
                             latent_query={"use_agent_type": False,
                                           "mode_emb": "none", # linear, mlp, add, none
                                           "mode_init": "xavier", # uniform, xavier
                                           "scale": 5.0},
                             n_latent_query=192,
                             n_encoder_layers=2)  

decoder = MotionDecoder()

In [4]:
f = None
femb = None
t = None
for batch in dl:
    print(batch['agent/pos'].shape)
    batch = prepreprocess(batch=batch)
    batch = preprocess(batch=batch)
    motion_tokens = torch.cat((batch["ac/target_pos"], batch["gt/pos"]), dim = -2)
    target_types = batch["ac/target_type"]
    t = motion_tokens
    print(motion_tokens.shape)
    input_dict = {
    k.split("input/")[-1]: v for k, v in batch.items() if "input/" in k
    }
    valid = input_dict["target_valid"].any(-1)
    target_emb, target_valid, other_emb, other_valid, map_emb, map_valid = inputproj(target_valid = input_dict["target_valid"], 
              target_attr = input_dict["target_attr"],
              other_valid = input_dict["other_valid"],
              other_attr = input_dict["other_attr"],
              map_valid = input_dict["map_valid"],
              map_attr = input_dict["map_attr"],)
    print(target_valid.shape, other_valid.shape, map_valid.shape)
    fused_emb, fused_emb_invalid = encoder(
                target_emb, target_valid, other_emb, other_valid, map_emb, map_valid, input_dict["target_type"], valid
            )
    episode = decoder(motion_tokens, target_types, fused_emb, fused_emb_invalid)
    f = (target_emb, target_valid, other_emb, other_valid, map_emb, map_valid)
    femb = fused_emb
    fmask = fused_emb_invalid
    break

torch.Size([2, 110, 64, 2])
torch.Size([2, 8, 110, 2])
torch.Size([16, 50]) torch.Size([16, 2400]) torch.Size([16, 10240])


In [5]:
episode.shape

torch.Size([1760, 256])

In [6]:
t.reshape((-1, 110, 2))[1] == t[0][1]

tensor([[True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [T

In [7]:
episode.shape

torch.Size([1760, 256])

In [8]:
episode[1] == episode.reshape((-1, 256))[110:220]

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [9]:
import numpy as np

t[1] == t.reshape((-1, 110, 2))[8:]

tensor([[[True, True],
         [True, True],
         [True, True],
         ...,
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         ...,
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         ...,
         [True, True],
         [True, True],
         [True, True]],

        ...,

        [[True, True],
         [True, True],
         [True, True],
         ...,
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         ...,
         [True, True],
         [True, True],
         [True, True]],

        [[True, True],
         [True, True],
         [True, True],
         ...,
         [True, True],
         [True, True],
         [True, True]]])

In [10]:
episode#.reshape(2, 8, 110, -1).shape#.unsqueeze(1)#.repeat(1, 6, 1, 1, 1).shape

tensor([[ 1.3078, -0.7891,  1.1052,  ..., -0.5589,  0.9723,  0.2858],
        [ 1.9711,  0.1538,  0.0097,  ...,  0.9854,  0.6055, -1.1061],
        [ 0.3877, -2.1134,  0.0943,  ...,  2.2777,  2.0868, -0.5535],
        ...,
        [-0.3336, -0.5869,  0.3268,  ...,  0.1246,  2.5931,  0.6938],
        [ 1.4502, -1.0639, -0.2516,  ...,  0.7508,  4.1432,  0.5801],
        [ 0.7104, -0.8970, -1.9450,  ...,  0.8618,  1.9894,  2.3208]],
       grad_fn=<ViewBackward0>)

In [11]:
episode.repeat(6, 1, 1)

tensor([[[ 1.3078, -0.7891,  1.1052,  ..., -0.5589,  0.9723,  0.2858],
         [ 1.9711,  0.1538,  0.0097,  ...,  0.9854,  0.6055, -1.1061],
         [ 0.3877, -2.1134,  0.0943,  ...,  2.2777,  2.0868, -0.5535],
         ...,
         [-0.3336, -0.5869,  0.3268,  ...,  0.1246,  2.5931,  0.6938],
         [ 1.4502, -1.0639, -0.2516,  ...,  0.7508,  4.1432,  0.5801],
         [ 0.7104, -0.8970, -1.9450,  ...,  0.8618,  1.9894,  2.3208]],

        [[ 1.3078, -0.7891,  1.1052,  ..., -0.5589,  0.9723,  0.2858],
         [ 1.9711,  0.1538,  0.0097,  ...,  0.9854,  0.6055, -1.1061],
         [ 0.3877, -2.1134,  0.0943,  ...,  2.2777,  2.0868, -0.5535],
         ...,
         [-0.3336, -0.5869,  0.3268,  ...,  0.1246,  2.5931,  0.6938],
         [ 1.4502, -1.0639, -0.2516,  ...,  0.7508,  4.1432,  0.5801],
         [ 0.7104, -0.8970, -1.9450,  ...,  0.8618,  1.9894,  2.3208]],

        [[ 1.3078, -0.7891,  1.1052,  ..., -0.5589,  0.9723,  0.2858],
         [ 1.9711,  0.1538,  0.0097,  ...,  0

In [12]:
fused_emb.shape

torch.Size([16, 192, 256])

In [13]:
input['target_valid'][1][5]

TypeError: 'method' object is not subscriptable

In [81]:
fmask.shape

torch.Size([6, 192])

In [79]:
177 * 2000

354000

In [4]:
from torch import nn
import torch

temp = torch.randn([18, 110, 256])
self_attn = nn.MultiheadAttention(embed_dim = 256, 
                                num_heads = 2,
                                dropout = 0.0,
                                batch_first=True)